# ROTATED RETINANET

In [ ]:
import os
import cv2
import torch
import mmrotate
import numpy as np
from mmdet.apis import init_detector, inference_detector

# -------------------- Path Setup --------------------
IMAGE_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\images\train\images"
OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_rrn"
CONFIG_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\mmrotate\configs\rotated_retinanet\rotated_retinanet_obb_r50_fpn_1x_dota_le135.py"
CHECKPOINT_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\rotated_retinanet_obb_r50_fpn_1x_dota_le135-e4131166.pth"
LOW_CONFIDENCE = 0.1

os.makedirs(OUTPUT_DIR, exist_ok=True)
TXT_FOLDER = os.path.join(OUTPUT_DIR, "txt_annotations")
os.makedirs(TXT_FOLDER, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {device}")

model = init_detector(CONFIG_FILE, CHECKPOINT_FILE, device=device)
class_names = model.CLASSES if hasattr(model, 'CLASSES') else model.dataset_meta['CLASSES']
print(f"✅ Model loaded with {len(class_names)} classes: {class_names}")

def draw_obb(img, obb, label, score, color=(0, 255, 0), thickness=2):
    cx, cy, w, h, angle = obb
    rect = ((cx, cy), (w, h), angle * 180 / np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, color, thickness)
    label_text = f"{class_names[label] if label < len(class_names) else 'No_Detection'}: {score:.2f}"
    cv2.putText(img, label_text, (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Gather image files
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    print(f"❌ No image files found in {IMAGE_DIR}")
else:
    print(f"✅ Found {len(image_files)} image(s) to process")

total_images = 0
images_with_detections = 0
skipped_images = []
class_detection_counts = {i: 0 for i in range(len(class_names))}
class_detection_counts["No_Detection"] = 0

for img_name in image_files:
    total_images += 1
    img_path = os.path.join(IMAGE_DIR, img_name)
    print(f"Processing {img_name}...")

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Failed to load image: {img_path}")
        skipped_images.append(img_name)
        continue

    result = inference_detector(model, img_path)

    drawn_any = False
    pred_data = []

    if isinstance(result, tuple):
        bbox_results, _ = result
        for class_id, bboxes in enumerate(bbox_results):
            for bbox in bboxes:
                score = bbox[5]
                obb = bbox[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })
    elif hasattr(result, 'pred_instances'):
        pred_instances = result.pred_instances
        bboxes = pred_instances.bboxes.cpu().numpy()
        scores = pred_instances.scores.cpu().numpy()
        labels = pred_instances.labels.cpu().numpy()
        for bbox, score, label in zip(bboxes, scores, labels):
            obb = bbox.tolist()
            if score >= LOW_CONFIDENCE:
                draw_obb(img, obb, label, score)
                drawn_any = True
                class_detection_counts[label.item()] += 1
                pred_data.append({
                    "obb": obb,
                    "score": float(score),
                    "label": int(label),
                    "label_name": class_names[label]
                })
    else:
        for class_id, class_result in enumerate(result):
            for res in class_result:
                score = res[-1]
                obb = res[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })

    # If no detection, draw dummy red box
    if not drawn_any:
        print(f"⚠️ No predictions for {img_name}, drawing dummy box.")
        h, w = img.shape[:2]
        dummy_cx, dummy_cy = w // 2, h // 2
        dummy_w, dummy_h = w // 5, h // 5
        dummy_angle = 0
        dummy_obb = [dummy_cx, dummy_cy, dummy_w, dummy_h, dummy_angle]
        dummy_score = 0.0
        draw_obb(img, dummy_obb, len(class_names), dummy_score, color=(0, 0, 255))
        pred_data.append({
            "obb": dummy_obb,
            "score": dummy_score,
            "label": len(class_names),
            "label_name": "No_Detection"
        })
        class_detection_counts["No_Detection"] += 1
    else:
        print(f"✅ Predictions drawn for {img_name}")
        images_with_detections += 1

    # Save image
    output_path = os.path.join(OUTPUT_DIR, img_name)
    cv2.imwrite(output_path, img)

    # Save .txt annotation
    txt_filename = os.path.splitext(img_name)[0] + ".txt"
    txt_path = os.path.join(TXT_FOLDER, txt_filename)

    with open(txt_path, 'w') as f:
        for pred in pred_data:
            cx, cy, w, h, angle = pred['obb']
            rect = ((cx, cy), (w, h), angle * 180 / np.pi)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            points_flat = box.flatten()
            coords = " ".join(map(str, points_flat))
            label_name = pred['label_name'].replace(" ", "_")  # Prevent space in label
            score = pred['score']
            line = f"{coords} {label_name} {score:.4f}\n"
            f.write(line)

    print(f"📄 TXT saved: {txt_filename}")

# ========== Summary ==========
print("\n===== Detection Summary =====")
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections} ({images_with_detections / total_images * 100:.1f}%)")
print(f"Images with dummy boxes: {total_images - images_with_detections} ({(total_images - images_with_detections) / total_images * 100:.1f}%)")

print("\n📊 Detections by class:")
for class_id, count in class_detection_counts.items():
    name = class_names[class_id] if isinstance(class_id, int) and class_id < len(class_names) else class_id
    if count > 0:
        print(f"  {name}: {count} detections")

if skipped_images:
    print("\n⚠️ Skipped the following images (could not load):")
    for name in skipped_images:
        print(f" - {name}")

print("\n🎉 All images processed and saved with OBBs and TXT annotations.")

d:\Anaconda\envs\cfi_net\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


✅ Using device: cuda
load checkpoint from local path: D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\rotated_retinanet_obb_r50_fpn_1x_dota_le135-e4131166.pth
✅ Model loaded with 15 classes: ('plane', 'baseball-diamond', 'bridge', 'ground-track-field', 'small-vehicle', 'large-vehicle', 'ship', 'tennis-court', 'basketball-court', 'storage-tank', 'soccer-ball-field', 'roundabout', 'harbor', 'swimming-pool', 'helicopter')
✅ Found 1411 image(s) to process
Processing P0000.png...
✅ Predictions drawn for P0000.png
📄 TXT saved: P0000.txt
Processing P0001.png...
✅ Predictions drawn for P0001.png
📄 TXT saved: P0001.txt
Processing P0002.png...
✅ Predictions drawn for P0002.png
📄 TXT saved: P0002.txt
Processing P0005.png...
✅ Predictions drawn for P0005.png
📄 TXT saved: P0005.txt
Processing P0008.png...
✅ Predictions drawn for P0008.png
📄 TXT saved: P0008.txt
Processing P0010.png...
✅ Predictions drawn for P0010.png
📄 TXT saved: P0010.txt
Processing P0011.png...
✅ Predictions drawn for P0011.png


In [ ]:
import os
import numpy as np
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import matplotlib.pyplot as plt

class OBBEvaluator:
    def __init__(self, gt_dir, pred_dir, iou_threshold=0.5, confidence_threshold=0.01):
        """
        Initialize the OBB evaluator
        
        Args:
            gt_dir: Directory containing ground truth text files
            pred_dir: Directory containing prediction text files
            iou_threshold: IoU threshold for considering a detection as true positive
            confidence_threshold: Minimum confidence score to consider a prediction
        """
        self.gt_dir = gt_dir
        self.pred_dir = pred_dir
        self.iou_threshold = iou_threshold
        self.confidence_threshold = confidence_threshold
        self.class_names = set()
        self.results = {}
        # Store all predictions for PR curves
        self.all_predictions = defaultdict(list)
        # Store class cumulative stats
        self.class_cumulative = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        # Count of missing prediction files
        self.missing_pred_files = 0
    
    def parse_gt_file(self, gt_file):
        """
        Parse ground truth file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name difficulty_score
        """
        gts = []
        with open(gt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + difficulty
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and ignore difficulty score
                    class_name = parts[8]
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    gts.append({
                        'polygon': points,
                        'class': class_name,
                        'detected': False  # Flag to mark if this gt is detected
                    })
                except Exception as e:
                    print(f"Error parsing GT line: {line}")
                    print(f"Exception: {e}")
        return gts
    
    def parse_pred_file(self, pred_file):
        """
        Parse prediction file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name confidence_score
        """
        preds = []
        if not os.path.exists(pred_file):
            return preds  # Return empty list if file doesn't exist
            
        with open(pred_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + confidence
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and confidence score
                    class_name = parts[8]
                    confidence = float(parts[9])
                    
                    # Skip predictions with confidence below threshold
                    if confidence < self.confidence_threshold:
                        continue
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    preds.append({
                        'polygon': points,
                        'class': class_name,
                        'confidence': confidence
                    })
                except Exception as e:
                    print(f"Error parsing prediction line: {line}")
                    print(f"Exception: {e}")
        
        # Sort predictions by confidence (descending)
        preds.sort(key=lambda x: x['confidence'], reverse=True)
        return preds
    
    def calculate_iou(self, poly1, poly2):
        """Calculate IoU between two polygons"""
        try:
            polygon1 = Polygon(poly1)
            polygon2 = Polygon(poly2)
            
            if not polygon1.is_valid or not polygon2.is_valid:
                return 0.0
            
            # Calculate intersection and union areas
            intersection_area = polygon1.intersection(polygon2).area
            union_area = polygon1.area + polygon2.area - intersection_area
            
            # Avoid division by zero
            if union_area == 0:
                return 0.0
                
            return intersection_area / union_area
        except Exception as e:
            print(f"Error calculating IoU: {e}")
            return 0.0
    
    def evaluate_image(self, gt_file, pred_file):
        """Evaluate a single image"""
        image_name = os.path.basename(gt_file).split('.')[0]
        
        # Parse GT and prediction files
        gts = self.parse_gt_file(gt_file)
        preds = self.parse_pred_file(pred_file)
        
        # Check if prediction file exists
        if not os.path.exists(pred_file):
            self.missing_pred_files += 1
            # Count all GTs as false negatives if no predictions
            for gt in gts:
                self.class_cumulative[gt['class']]['FN'] += 1
            return
        
        # Initialize per-class results for this image
        class_results = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        class_predictions = defaultdict(list)
        
        # For each prediction, find the best matching ground truth
        for pred in preds:
            pred_class = pred['class']
            pred_poly = pred['polygon']
            pred_conf = pred['confidence']
            
            best_iou = 0
            best_gt_idx = -1
            
            # Find the best matching GT of the same class
            for idx, gt in enumerate(gts):
                if gt['class'] != pred_class or gt['detected']:
                    continue
                
                iou = self.calculate_iou(pred_poly, gt['polygon'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            # Record as TP or FP based on IoU threshold
            if best_gt_idx >= 0 and best_iou >= self.iou_threshold:
                class_predictions[pred_class].append((pred_conf, 1))  # True positive
                gts[best_gt_idx]['detected'] = True
                class_results[pred_class]['TP'] += 1
                self.class_cumulative[pred_class]['TP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 1))
            else:
                class_predictions[pred_class].append((pred_conf, 0))  # False positive
                class_results[pred_class]['FP'] += 1
                self.class_cumulative[pred_class]['FP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 0))
        
        # Count false negatives (undetected GTs)
        for gt in gts:
            if not gt['detected']:
                class_results[gt['class']]['FN'] += 1
                self.class_cumulative[gt['class']]['FN'] += 1
    
    def calculate_precision_recall(self, tp, fp, fn):
        """Calculate precision and recall"""
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        return precision, recall
    
    def calculate_f1(self, precision, recall):
        """Calculate F1 score"""
        return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    def calculate_ap(self, precisions, recalls):
        """Calculate Average Precision using 11-point interpolation"""
        ap = 0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11
        return ap
    
    def evaluate(self):
        """Evaluate all images and calculate metrics"""
        # Get list of files
        gt_files = sorted([f for f in os.listdir(self.gt_dir) if f.endswith('.txt')])
        print(f"Found {len(gt_files)} ground truth files")
        
        # Process each image
        for gt_file in gt_files:
            image_id = os.path.splitext(gt_file)[0]
            pred_file = image_id + '.txt'
            
            gt_path = os.path.join(self.gt_dir, gt_file)
            pred_path = os.path.join(self.pred_dir, pred_file)
            
            # Skip if prediction file doesn't exist but still record the missing files
            if not os.path.exists(pred_path):
                self.evaluate_image(gt_path, pred_path)  # This will handle missing files
                continue
            
            # Evaluate the image
            self.evaluate_image(gt_path, pred_path)
            
        if self.missing_pred_files > 0:
            print(f"\nWarning: {self.missing_pred_files} out of {len(gt_files)} prediction files were missing.")
            print(f"These are counted as having no detections (all ground truths counted as false negatives).")
        
        # Calculate metrics for each class
        self.results = {}
        mean_ap = 0
        processed_classes = 0
        
        for class_name in self.class_names:
            TP = self.class_cumulative[class_name]['TP']
            FP = self.class_cumulative[class_name]['FP']
            FN = self.class_cumulative[class_name]['FN']
            
            precision, recall = self.calculate_precision_recall(TP, FP, FN)
            f1 = self.calculate_f1(precision, recall)
            
            # Sort predictions by confidence for AP calculation
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            # Skip AP calculation if no ground truths or predictions
            if TP + FN == 0 or len(class_preds) == 0:
                ap = 0
            else:
                cumulative_tp = 0
                cumulative_fp = 0
                precisions = []
                recalls = []
                
                for conf, is_tp in class_preds:
                    if is_tp:
                        cumulative_tp += 1
                    else:
                        cumulative_fp += 1
                    
                    prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                    rec = cumulative_tp / (TP + FN)
                    
                    precisions.append(prec)
                    recalls.append(rec)
                
                precisions = np.array(precisions)
                recalls = np.array(recalls)
                ap = self.calculate_ap(precisions, recalls)
                processed_classes += 1
            
            self.results[class_name] = {
                'AP': ap,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'TP': TP,
                'FP': FP,
                'FN': FN
            }
            
            mean_ap += ap
        
        # Calculate mAP only over classes with predictions
        if processed_classes > 0:
            mean_ap /= processed_classes
        else:
            mean_ap = 0
        
        self.results['mAP'] = mean_ap
        
        return self.results
    
    def plot_precision_recall_curves(self, output_dir):
        """Plot precision-recall curves for all classes"""
        os.makedirs(output_dir, exist_ok=True)
        
        plt.figure(figsize=(10, 8))
        
        for class_name in sorted(self.class_names):
            # Skip classes with no predictions
            if class_name not in self.all_predictions or len(self.all_predictions[class_name]) == 0:
                continue
            
            # Sort predictions by confidence
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            TP = self.class_cumulative[class_name]['TP']
            FN = self.class_cumulative[class_name]['FN']
            
            # Skip if no ground truths
            if TP + FN == 0:
                continue
            
            # Calculate cumulative precision and recall
            cumulative_tp = 0
            cumulative_fp = 0
            precisions = []
            recalls = []
            
            for conf, is_tp in class_preds:
                if is_tp:
                    cumulative_tp += 1
                else:
                    cumulative_fp += 1
                
                prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                rec = cumulative_tp / (TP + FN)
                
                precisions.append(prec)
                recalls.append(rec)
            
            plt.plot(recalls, precisions, label=f"{class_name} (AP={self.results[class_name]['AP']:.3f})")
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'precision_recall_curves_RNN.png'))
        plt.close()
    
    def print_results(self):
        """Print evaluation results"""
        print("\n===== EVALUATION RESULTS =====")
        print(f"IoU Threshold: {self.iou_threshold}")
        print(f"Confidence Threshold: {self.confidence_threshold}")
        print(f"mAP: {self.results['mAP']:.4f}")
        print("\nPer-class results:")
        
        # Print header
        print(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}")
        print('-' * 70)
        
        # Sort classes for consistent output
        for class_name in sorted(self.class_names):
            r = self.results[class_name]
            print(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}")
        
        print("=" * 70)
    
    def save_results_to_file(self, output_path):
        """Save results to file"""
        with open(output_path, 'w') as f:
            f.write("===== EVALUATION RESULTS =====\n")
            f.write(f"IoU Threshold: {self.iou_threshold}\n")
            f.write(f"Confidence Threshold: {self.confidence_threshold}\n")
            f.write(f"mAP: {self.results['mAP']:.4f}\n\n")
            
            # if self.missing_pred_files > 0:
            #     f.write(f"Note: {self.missing_pred_files} prediction files were missing and counted as no detections.\n\n")
                
            f.write("Per-class results:\n")
            
            # Print header
            f.write(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}\n")
            f.write('-' * 70 + '\n')
            
            # Sort classes for consistent output
            for class_name in sorted(self.class_names):
                r = self.results[class_name]
                f.write(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}\n")
            
            f.write("=" * 70 + '\n')


# Usage example
if __name__ == "__main__":
    # Define paths
    GT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\labels\train\labelTxt-v1.0\labelTxt"  # Ground truth directory
    PRED_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_rrn1\txt_annotations"  # Prediction directory
    OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\evaluation_results"  # Output directory
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Create evaluator with default parameters
    evaluator = OBBEvaluator(
        gt_dir=GT_DIR,
        pred_dir=PRED_DIR,
        iou_threshold=0.5,  # IoU threshold for TP/FP determination
        confidence_threshold=0.01  # Min confidence to consider a prediction
    )
    
    # Run evaluation
    results = evaluator.evaluate()
    
    # Print results
    evaluator.print_results()
    
    # Save results to file
    evaluator.save_results_to_file(os.path.join(OUTPUT_DIR, "evaluation_results_RRN.txt"))
    
    # Generate PR curves
    try:
        evaluator.plot_precision_recall_curves(OUTPUT_DIR)
        print(f"Precision-Recall curves saved to {OUTPUT_DIR}")
    except Exception as e:
        print(f"Error generating PR curves: {e}")
    
    print(f"Evaluation completed! Results saved to {OUTPUT_DIR}")

Found 1411 ground truth files

===== EVALUATION RESULTS =====
IoU Threshold: 0.5
Confidence Threshold: 0.01
mAP: 0.3425

Per-class results:
Class           AP       Precision  Recall     F1       TP     FP     FN    
----------------------------------------------------------------------
baseball-diamond 0.4470   0.4447     0.4458     0.4452   185    231    230   
basketball-court 0.5250   0.3199     0.5845     0.4135   301    640    214   
bridge          0.0909   0.1265     0.0850     0.1017   174    1201   1873  
ground-track-field 0.3919   0.2491     0.4215     0.3131   137    413    188   
harbor          0.2276   0.3230     0.2863     0.3036   1713   3590   4270  
helicopter      0.0909   0.0579     0.0492     0.0532   31     504    599   
large-vehicle   0.4797   0.5345     0.5357     0.5351   9090   7917   7879  
plane           0.4205   0.2562     0.4236     0.3193   3412   9904   4643  
roundabout      0.2760   0.2112     0.3108     0.2515   124    463    275   
ship          

# ORIENTED RCNN

In [1]:
import os
import cv2
import torch
import mmrotate
import numpy as np
from mmdet.apis import init_detector, inference_detector

# -------------------- Path Setup --------------------
IMAGE_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\images\train\images"
OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_orcnn"
CONFIG_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\mmrotate\configs\oriented_rcnn\oriented_rcnn_r50_fpn_fp16_1x_dota_le90.py"
CHECKPOINT_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\oriented_rcnn_r50_fpn_fp16_1x_dota_le90-57c88621.pth"
LOW_CONFIDENCE = 0.1

os.makedirs(OUTPUT_DIR, exist_ok=True)
TXT_FOLDER = os.path.join(OUTPUT_DIR, "txt_annotations")
os.makedirs(TXT_FOLDER, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {device}")

model = init_detector(CONFIG_FILE, CHECKPOINT_FILE, device=device)
class_names = model.CLASSES if hasattr(model, 'CLASSES') else model.dataset_meta['CLASSES']
print(f"✅ Model loaded with {len(class_names)} classes: {class_names}")

def draw_obb(img, obb, label, score, color=(0, 255, 0), thickness=2):
    cx, cy, w, h, angle = obb
    rect = ((cx, cy), (w, h), angle * 180 / np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, color, thickness)
    label_text = f"{class_names[label] if label < len(class_names) else 'No_Detection'}: {score:.2f}"
    cv2.putText(img, label_text, (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Gather image files
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    print(f"❌ No image files found in {IMAGE_DIR}")
else:
    print(f"✅ Found {len(image_files)} image(s) to process")

total_images = 0
images_with_detections = 0
skipped_images = []
class_detection_counts = {i: 0 for i in range(len(class_names))}
class_detection_counts["No_Detection"] = 0

for img_name in image_files:
    total_images += 1
    img_path = os.path.join(IMAGE_DIR, img_name)
    print(f"Processing {img_name}...")

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Failed to load image: {img_path}")
        skipped_images.append(img_name)
        continue

    result = inference_detector(model, img_path)

    drawn_any = False
    pred_data = []

    if isinstance(result, tuple):
        bbox_results, _ = result
        for class_id, bboxes in enumerate(bbox_results):
            for bbox in bboxes:
                score = bbox[5]
                obb = bbox[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })
    elif hasattr(result, 'pred_instances'):
        pred_instances = result.pred_instances
        bboxes = pred_instances.bboxes.cpu().numpy()
        scores = pred_instances.scores.cpu().numpy()
        labels = pred_instances.labels.cpu().numpy()
        for bbox, score, label in zip(bboxes, scores, labels):
            obb = bbox.tolist()
            if score >= LOW_CONFIDENCE:
                draw_obb(img, obb, label, score)
                drawn_any = True
                class_detection_counts[label.item()] += 1
                pred_data.append({
                    "obb": obb,
                    "score": float(score),
                    "label": int(label),
                    "label_name": class_names[label]
                })
    else:
        for class_id, class_result in enumerate(result):
            for res in class_result:
                score = res[-1]
                obb = res[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })

    # If no detection, draw dummy red box
    if not drawn_any:
        print(f"⚠️ No predictions for {img_name}, drawing dummy box.")
        h, w = img.shape[:2]
        dummy_cx, dummy_cy = w // 2, h // 2
        dummy_w, dummy_h = w // 5, h // 5
        dummy_angle = 0
        dummy_obb = [dummy_cx, dummy_cy, dummy_w, dummy_h, dummy_angle]
        dummy_score = 0.0
        draw_obb(img, dummy_obb, len(class_names), dummy_score, color=(0, 0, 255))
        pred_data.append({
            "obb": dummy_obb,
            "score": dummy_score,
            "label": len(class_names),
            "label_name": "No_Detection"
        })
        class_detection_counts["No_Detection"] += 1
    else:
        print(f"✅ Predictions drawn for {img_name}")
        images_with_detections += 1

    # Save image
    output_path = os.path.join(OUTPUT_DIR, img_name)
    cv2.imwrite(output_path, img)

    # Save .txt annotation
    txt_filename = os.path.splitext(img_name)[0] + ".txt"
    txt_path = os.path.join(TXT_FOLDER, txt_filename)

    with open(txt_path, 'w') as f:
        for pred in pred_data:
            cx, cy, w, h, angle = pred['obb']
            rect = ((cx, cy), (w, h), angle * 180 / np.pi)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            points_flat = box.flatten()
            coords = " ".join(map(str, points_flat))
            label_name = pred['label_name'].replace(" ", "_")  # Prevent space in label
            score = pred['score']
            line = f"{coords} {label_name} {score:.4f}\n"
            f.write(line)

    print(f"📄 TXT saved: {txt_filename}")

# ========== Summary ==========
print("\n===== Detection Summary =====")
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections} ({images_with_detections / total_images * 100:.1f}%)")
print(f"Images with dummy boxes: {total_images - images_with_detections} ({(total_images - images_with_detections) / total_images * 100:.1f}%)")

print("\n📊 Detections by class:")
for class_id, count in class_detection_counts.items():
    name = class_names[class_id] if isinstance(class_id, int) and class_id < len(class_names) else class_id
    if count > 0:
        print(f"  {name}: {count} detections")

if skipped_images:
    print("\n⚠️ Skipped the following images (could not load):")
    for name in skipped_images:
        print(f" - {name}")

print("\n🎉 All images processed and saved with OBBs and TXT annotations.")

d:\Anaconda\envs\cfi_net\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


✅ Using device: cuda


d:\Anaconda\envs\cfi_net\lib\site-packages\mmdet\models\dense_heads\anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\oriented_rcnn_r50_fpn_fp16_1x_dota_le90-57c88621.pth
✅ Model loaded with 15 classes: ('plane', 'baseball-diamond', 'bridge', 'ground-track-field', 'small-vehicle', 'large-vehicle', 'ship', 'tennis-court', 'basketball-court', 'storage-tank', 'soccer-ball-field', 'roundabout', 'harbor', 'swimming-pool', 'helicopter')
✅ Found 1411 image(s) to process
Processing P0000.png...


d:\Anaconda\envs\cfi_net\lib\site-packages\mmdet\models\dense_heads\anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


✅ Predictions drawn for P0000.png
📄 TXT saved: P0000.txt
Processing P0001.png...
✅ Predictions drawn for P0001.png
📄 TXT saved: P0001.txt
Processing P0002.png...
✅ Predictions drawn for P0002.png
📄 TXT saved: P0002.txt
Processing P0005.png...
✅ Predictions drawn for P0005.png
📄 TXT saved: P0005.txt
Processing P0008.png...
✅ Predictions drawn for P0008.png
📄 TXT saved: P0008.txt
Processing P0010.png...
✅ Predictions drawn for P0010.png
📄 TXT saved: P0010.txt
Processing P0011.png...
✅ Predictions drawn for P0011.png
📄 TXT saved: P0011.txt
Processing P0012.png...
✅ Predictions drawn for P0012.png
📄 TXT saved: P0012.txt
Processing P0013.png...
✅ Predictions drawn for P0013.png
📄 TXT saved: P0013.txt
Processing P0018.png...
✅ Predictions drawn for P0018.png
📄 TXT saved: P0018.txt
Processing P0020.png...
✅ Predictions drawn for P0020.png
📄 TXT saved: P0020.txt
Processing P0021.png...
✅ Predictions drawn for P0021.png
📄 TXT saved: P0021.txt
Processing P0022.png...
✅ Predictions drawn for P002

In [2]:
import os
import numpy as np
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import matplotlib.pyplot as plt

class OBBEvaluator:
    def __init__(self, gt_dir, pred_dir, iou_threshold=0.5, confidence_threshold=0.01):
        """
        Initialize the OBB evaluator
        
        Args:
            gt_dir: Directory containing ground truth text files
            pred_dir: Directory containing prediction text files
            iou_threshold: IoU threshold for considering a detection as true positive
            confidence_threshold: Minimum confidence score to consider a prediction
        """
        self.gt_dir = gt_dir
        self.pred_dir = pred_dir
        self.iou_threshold = iou_threshold
        self.confidence_threshold = confidence_threshold
        self.class_names = set()
        self.results = {}
        # Store all predictions for PR curves
        self.all_predictions = defaultdict(list)
        # Store class cumulative stats
        self.class_cumulative = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        # Count of missing prediction files
        self.missing_pred_files = 0
    
    def parse_gt_file(self, gt_file):
        """
        Parse ground truth file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name difficulty_score
        """
        gts = []
        with open(gt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + difficulty
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and ignore difficulty score
                    class_name = parts[8]
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    gts.append({
                        'polygon': points,
                        'class': class_name,
                        'detected': False  # Flag to mark if this gt is detected
                    })
                except Exception as e:
                    print(f"Error parsing GT line: {line}")
                    print(f"Exception: {e}")
        return gts
    
    def parse_pred_file(self, pred_file):
        """
        Parse prediction file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name confidence_score
        """
        preds = []
        if not os.path.exists(pred_file):
            return preds  # Return empty list if file doesn't exist
            
        with open(pred_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + confidence
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and confidence score
                    class_name = parts[8]
                    confidence = float(parts[9])
                    
                    # Skip predictions with confidence below threshold
                    if confidence < self.confidence_threshold:
                        continue
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    preds.append({
                        'polygon': points,
                        'class': class_name,
                        'confidence': confidence
                    })
                except Exception as e:
                    print(f"Error parsing prediction line: {line}")
                    print(f"Exception: {e}")
        
        # Sort predictions by confidence (descending)
        preds.sort(key=lambda x: x['confidence'], reverse=True)
        return preds
    
    def calculate_iou(self, poly1, poly2):
        """Calculate IoU between two polygons"""
        try:
            polygon1 = Polygon(poly1)
            polygon2 = Polygon(poly2)
            
            if not polygon1.is_valid or not polygon2.is_valid:
                return 0.0
            
            # Calculate intersection and union areas
            intersection_area = polygon1.intersection(polygon2).area
            union_area = polygon1.area + polygon2.area - intersection_area
            
            # Avoid division by zero
            if union_area == 0:
                return 0.0
                
            return intersection_area / union_area
        except Exception as e:
            print(f"Error calculating IoU: {e}")
            return 0.0
    
    def evaluate_image(self, gt_file, pred_file):
        """Evaluate a single image"""
        image_name = os.path.basename(gt_file).split('.')[0]
        
        # Parse GT and prediction files
        gts = self.parse_gt_file(gt_file)
        preds = self.parse_pred_file(pred_file)
        
        # Check if prediction file exists
        if not os.path.exists(pred_file):
            self.missing_pred_files += 1
            # Count all GTs as false negatives if no predictions
            for gt in gts:
                self.class_cumulative[gt['class']]['FN'] += 1
            return
        
        # Initialize per-class results for this image
        class_results = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        class_predictions = defaultdict(list)
        
        # For each prediction, find the best matching ground truth
        for pred in preds:
            pred_class = pred['class']
            pred_poly = pred['polygon']
            pred_conf = pred['confidence']
            
            best_iou = 0
            best_gt_idx = -1
            
            # Find the best matching GT of the same class
            for idx, gt in enumerate(gts):
                if gt['class'] != pred_class or gt['detected']:
                    continue
                
                iou = self.calculate_iou(pred_poly, gt['polygon'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            # Record as TP or FP based on IoU threshold
            if best_gt_idx >= 0 and best_iou >= self.iou_threshold:
                class_predictions[pred_class].append((pred_conf, 1))  # True positive
                gts[best_gt_idx]['detected'] = True
                class_results[pred_class]['TP'] += 1
                self.class_cumulative[pred_class]['TP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 1))
            else:
                class_predictions[pred_class].append((pred_conf, 0))  # False positive
                class_results[pred_class]['FP'] += 1
                self.class_cumulative[pred_class]['FP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 0))
        
        # Count false negatives (undetected GTs)
        for gt in gts:
            if not gt['detected']:
                class_results[gt['class']]['FN'] += 1
                self.class_cumulative[gt['class']]['FN'] += 1
    
    def calculate_precision_recall(self, tp, fp, fn):
        """Calculate precision and recall"""
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        return precision, recall
    
    def calculate_f1(self, precision, recall):
        """Calculate F1 score"""
        return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    def calculate_ap(self, precisions, recalls):
        """Calculate Average Precision using 11-point interpolation"""
        ap = 0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11
        return ap
    
    def evaluate(self):
        """Evaluate all images and calculate metrics"""
        # Get list of files
        gt_files = sorted([f for f in os.listdir(self.gt_dir) if f.endswith('.txt')])
        print(f"Found {len(gt_files)} ground truth files")
        
        # Process each image
        for gt_file in gt_files:
            image_id = os.path.splitext(gt_file)[0]
            pred_file = image_id + '.txt'
            
            gt_path = os.path.join(self.gt_dir, gt_file)
            pred_path = os.path.join(self.pred_dir, pred_file)
            
            # Skip if prediction file doesn't exist but still record the missing files
            if not os.path.exists(pred_path):
                self.evaluate_image(gt_path, pred_path)  # This will handle missing files
                continue
            
            # Evaluate the image
            self.evaluate_image(gt_path, pred_path)
            
        if self.missing_pred_files > 0:
            print(f"\nWarning: {self.missing_pred_files} out of {len(gt_files)} prediction files were missing.")
            print(f"These are counted as having no detections (all ground truths counted as false negatives).")
        
        # Calculate metrics for each class
        self.results = {}
        mean_ap = 0
        processed_classes = 0
        
        for class_name in self.class_names:
            TP = self.class_cumulative[class_name]['TP']
            FP = self.class_cumulative[class_name]['FP']
            FN = self.class_cumulative[class_name]['FN']
            
            precision, recall = self.calculate_precision_recall(TP, FP, FN)
            f1 = self.calculate_f1(precision, recall)
            
            # Sort predictions by confidence for AP calculation
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            # Skip AP calculation if no ground truths or predictions
            if TP + FN == 0 or len(class_preds) == 0:
                ap = 0
            else:
                cumulative_tp = 0
                cumulative_fp = 0
                precisions = []
                recalls = []
                
                for conf, is_tp in class_preds:
                    if is_tp:
                        cumulative_tp += 1
                    else:
                        cumulative_fp += 1
                    
                    prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                    rec = cumulative_tp / (TP + FN)
                    
                    precisions.append(prec)
                    recalls.append(rec)
                
                precisions = np.array(precisions)
                recalls = np.array(recalls)
                ap = self.calculate_ap(precisions, recalls)
                processed_classes += 1
            
            self.results[class_name] = {
                'AP': ap,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'TP': TP,
                'FP': FP,
                'FN': FN
            }
            
            mean_ap += ap
        
        # Calculate mAP only over classes with predictions
        if processed_classes > 0:
            mean_ap /= processed_classes
        else:
            mean_ap = 0
        
        self.results['mAP'] = mean_ap
        
        return self.results
    
    def plot_precision_recall_curves(self, output_dir):
        """Plot precision-recall curves for all classes"""
        os.makedirs(output_dir, exist_ok=True)
        
        plt.figure(figsize=(10, 8))
        
        for class_name in sorted(self.class_names):
            # Skip classes with no predictions
            if class_name not in self.all_predictions or len(self.all_predictions[class_name]) == 0:
                continue
            
            # Sort predictions by confidence
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            TP = self.class_cumulative[class_name]['TP']
            FN = self.class_cumulative[class_name]['FN']
            
            # Skip if no ground truths
            if TP + FN == 0:
                continue
            
            # Calculate cumulative precision and recall
            cumulative_tp = 0
            cumulative_fp = 0
            precisions = []
            recalls = []
            
            for conf, is_tp in class_preds:
                if is_tp:
                    cumulative_tp += 1
                else:
                    cumulative_fp += 1
                
                prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                rec = cumulative_tp / (TP + FN)
                
                precisions.append(prec)
                recalls.append(rec)
            
            plt.plot(recalls, precisions, label=f"{class_name} (AP={self.results[class_name]['AP']:.3f})")
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'precision_recall_curves_ORCNN.png'))
        plt.close()
    
    def print_results(self):
        """Print evaluation results"""
        print("\n===== EVALUATION RESULTS =====")
        print(f"IoU Threshold: {self.iou_threshold}")
        print(f"Confidence Threshold: {self.confidence_threshold}")
        print(f"mAP: {self.results['mAP']:.4f}")
        print("\nPer-class results:")
        
        # Print header
        print(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}")
        print('-' * 70)
        
        # Sort classes for consistent output
        for class_name in sorted(self.class_names):
            r = self.results[class_name]
            print(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}")
        
        print("=" * 70)
    
    def save_results_to_file(self, output_path):
        """Save results to file"""
        with open(output_path, 'w') as f:
            f.write("===== EVALUATION RESULTS =====\n")
            f.write(f"IoU Threshold: {self.iou_threshold}\n")
            f.write(f"Confidence Threshold: {self.confidence_threshold}\n")
            f.write(f"mAP: {self.results['mAP']:.4f}\n\n")
            
            # if self.missing_pred_files > 0:
            #     f.write(f"Note: {self.missing_pred_files} prediction files were missing and counted as no detections.\n\n")
                
            f.write("Per-class results:\n")
            
            # Print header
            f.write(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}\n")
            f.write('-' * 70 + '\n')
            
            # Sort classes for consistent output
            for class_name in sorted(self.class_names):
                r = self.results[class_name]
                f.write(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}\n")
            
            f.write("=" * 70 + '\n')


# Usage example
if __name__ == "__main__":
    # Define paths
    GT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\labels\train\labelTxt-v1.0\labelTxt"  # Ground truth directory
    PRED_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_orcnn\txt_annotations"  # Prediction directory
    OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\evaluation_results"  # Output directory
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Create evaluator with default parameters
    evaluator = OBBEvaluator(
        gt_dir=GT_DIR,
        pred_dir=PRED_DIR,
        iou_threshold=0.5,  # IoU threshold for TP/FP determination
        confidence_threshold=0.01  # Min confidence to consider a prediction
    )
    
    # Run evaluation
    results = evaluator.evaluate()
    
    # Print results
    evaluator.print_results()
    
    # Save results to file
    evaluator.save_results_to_file(os.path.join(OUTPUT_DIR, "evaluation_results_ORCNN.txt"))
    
    # Generate PR curves
    try:
        evaluator.plot_precision_recall_curves(OUTPUT_DIR)
        print(f"Precision-Recall curves saved to {OUTPUT_DIR}")
    except Exception as e:
        print(f"Error generating PR curves: {e}")
    
    print(f"Evaluation completed! Results saved to {OUTPUT_DIR}")

Found 1411 ground truth files

===== EVALUATION RESULTS =====
IoU Threshold: 0.5
Confidence Threshold: 0.01
mAP: 0.4070

Per-class results:
Class           AP       Precision  Recall     F1       TP     FP     FN    
----------------------------------------------------------------------
baseball-diamond 0.4495   0.8341     0.4482     0.5831   186    37     229   
basketball-court 0.6236   0.7235     0.6350     0.6763   327    125    188   
bridge          0.0909   0.3731     0.0826     0.1352   169    284    1878  
ground-track-field 0.4484   0.6866     0.4246     0.5247   138    63     187   
harbor          0.4059   0.7073     0.4172     0.5248   2496   1033   3487  
helicopter      0.0909   0.1667     0.0587     0.0869   37     185    593   
large-vehicle   0.6143   0.7995     0.6385     0.7100   10835  2718   6134  
plane           0.5369   0.8741     0.5239     0.6551   4220   608    3835  
roundabout      0.2434   0.4573     0.2682     0.3381   107    127    292   
ship          

# ROTATED FASTER RCNN

In [3]:
import os
import cv2
import torch
import mmrotate
import numpy as np
from mmdet.apis import init_detector, inference_detector

# -------------------- Path Setup --------------------
IMAGE_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\images\train\images"
OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_rfrcnn"
CONFIG_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\mmrotate\configs\rotated_faster_rcnn\rotated_faster_rcnn_r50_fpn_1x_dota_le90.py"
CHECKPOINT_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\rotated_faster_rcnn_r50_fpn_1x_dota_le90-0393aa5c.pth"
LOW_CONFIDENCE = 0.1

os.makedirs(OUTPUT_DIR, exist_ok=True)
TXT_FOLDER = os.path.join(OUTPUT_DIR, "txt_annotations")
os.makedirs(TXT_FOLDER, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {device}")

model = init_detector(CONFIG_FILE, CHECKPOINT_FILE, device=device)
class_names = model.CLASSES if hasattr(model, 'CLASSES') else model.dataset_meta['CLASSES']
print(f"✅ Model loaded with {len(class_names)} classes: {class_names}")

def draw_obb(img, obb, label, score, color=(0, 255, 0), thickness=2):
    cx, cy, w, h, angle = obb
    rect = ((cx, cy), (w, h), angle * 180 / np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, color, thickness)
    label_text = f"{class_names[label] if label < len(class_names) else 'No_Detection'}: {score:.2f}"
    cv2.putText(img, label_text, (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Gather image files
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    print(f"❌ No image files found in {IMAGE_DIR}")
else:
    print(f"✅ Found {len(image_files)} image(s) to process")

total_images = 0
images_with_detections = 0
skipped_images = []
class_detection_counts = {i: 0 for i in range(len(class_names))}
class_detection_counts["No_Detection"] = 0

for img_name in image_files:
    total_images += 1
    img_path = os.path.join(IMAGE_DIR, img_name)
    print(f"Processing {img_name}...")

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Failed to load image: {img_path}")
        skipped_images.append(img_name)
        continue

    result = inference_detector(model, img_path)

    drawn_any = False
    pred_data = []

    if isinstance(result, tuple):
        bbox_results, _ = result
        for class_id, bboxes in enumerate(bbox_results):
            for bbox in bboxes:
                score = bbox[5]
                obb = bbox[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })
    elif hasattr(result, 'pred_instances'):
        pred_instances = result.pred_instances
        bboxes = pred_instances.bboxes.cpu().numpy()
        scores = pred_instances.scores.cpu().numpy()
        labels = pred_instances.labels.cpu().numpy()
        for bbox, score, label in zip(bboxes, scores, labels):
            obb = bbox.tolist()
            if score >= LOW_CONFIDENCE:
                draw_obb(img, obb, label, score)
                drawn_any = True
                class_detection_counts[label.item()] += 1
                pred_data.append({
                    "obb": obb,
                    "score": float(score),
                    "label": int(label),
                    "label_name": class_names[label]
                })
    else:
        for class_id, class_result in enumerate(result):
            for res in class_result:
                score = res[-1]
                obb = res[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })

    # If no detection, draw dummy red box
    if not drawn_any:
        print(f"⚠️ No predictions for {img_name}, drawing dummy box.")
        h, w = img.shape[:2]
        dummy_cx, dummy_cy = w // 2, h // 2
        dummy_w, dummy_h = w // 5, h // 5
        dummy_angle = 0
        dummy_obb = [dummy_cx, dummy_cy, dummy_w, dummy_h, dummy_angle]
        dummy_score = 0.0
        draw_obb(img, dummy_obb, len(class_names), dummy_score, color=(0, 0, 255))
        pred_data.append({
            "obb": dummy_obb,
            "score": dummy_score,
            "label": len(class_names),
            "label_name": "No_Detection"
        })
        class_detection_counts["No_Detection"] += 1
    else:
        print(f"✅ Predictions drawn for {img_name}")
        images_with_detections += 1

    # Save image
    output_path = os.path.join(OUTPUT_DIR, img_name)
    cv2.imwrite(output_path, img)

    # Save .txt annotation
    txt_filename = os.path.splitext(img_name)[0] + ".txt"
    txt_path = os.path.join(TXT_FOLDER, txt_filename)

    with open(txt_path, 'w') as f:
        for pred in pred_data:
            cx, cy, w, h, angle = pred['obb']
            rect = ((cx, cy), (w, h), angle * 180 / np.pi)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            points_flat = box.flatten()
            coords = " ".join(map(str, points_flat))
            label_name = pred['label_name'].replace(" ", "_")  # Prevent space in label
            score = pred['score']
            line = f"{coords} {label_name} {score:.4f}\n"
            f.write(line)

    print(f"📄 TXT saved: {txt_filename}")

# ========== Summary ==========
print("\n===== Detection Summary =====")
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections} ({images_with_detections / total_images * 100:.1f}%)")
print(f"Images with dummy boxes: {total_images - images_with_detections} ({(total_images - images_with_detections) / total_images * 100:.1f}%)")

print("\n📊 Detections by class:")
for class_id, count in class_detection_counts.items():
    name = class_names[class_id] if isinstance(class_id, int) and class_id < len(class_names) else class_id
    if count > 0:
        print(f"  {name}: {count} detections")

if skipped_images:
    print("\n⚠️ Skipped the following images (could not load):")
    for name in skipped_images:
        print(f" - {name}")

print("\n🎉 All images processed and saved with OBBs and TXT annotations.")

✅ Using device: cuda


d:\Anaconda\envs\cfi_net\lib\site-packages\mmdet\models\dense_heads\anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\rotated_faster_rcnn_r50_fpn_1x_dota_le90-0393aa5c.pth
✅ Model loaded with 15 classes: ('plane', 'baseball-diamond', 'bridge', 'ground-track-field', 'small-vehicle', 'large-vehicle', 'ship', 'tennis-court', 'basketball-court', 'storage-tank', 'soccer-ball-field', 'roundabout', 'harbor', 'swimming-pool', 'helicopter')
✅ Found 1411 image(s) to process
Processing P0000.png...


d:\Anaconda\envs\cfi_net\lib\site-packages\mmdet\models\dense_heads\anchor_head.py:123: UserWarning: DeprecationWarning: anchor_generator is deprecated, please use "prior_generator" instead
  warnings.warn('DeprecationWarning: anchor_generator is deprecated, '


✅ Predictions drawn for P0000.png
📄 TXT saved: P0000.txt
Processing P0001.png...
✅ Predictions drawn for P0001.png
📄 TXT saved: P0001.txt
Processing P0002.png...
✅ Predictions drawn for P0002.png
📄 TXT saved: P0002.txt
Processing P0005.png...
✅ Predictions drawn for P0005.png
📄 TXT saved: P0005.txt
Processing P0008.png...
✅ Predictions drawn for P0008.png
📄 TXT saved: P0008.txt
Processing P0010.png...
✅ Predictions drawn for P0010.png
📄 TXT saved: P0010.txt
Processing P0011.png...
✅ Predictions drawn for P0011.png
📄 TXT saved: P0011.txt
Processing P0012.png...
✅ Predictions drawn for P0012.png
📄 TXT saved: P0012.txt
Processing P0013.png...
✅ Predictions drawn for P0013.png
📄 TXT saved: P0013.txt
Processing P0018.png...
✅ Predictions drawn for P0018.png
📄 TXT saved: P0018.txt
Processing P0020.png...
✅ Predictions drawn for P0020.png
📄 TXT saved: P0020.txt
Processing P0021.png...
✅ Predictions drawn for P0021.png
📄 TXT saved: P0021.txt
Processing P0022.png...
✅ Predictions drawn for P002

In [4]:
import os
import numpy as np
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import matplotlib.pyplot as plt

class OBBEvaluator:
    def __init__(self, gt_dir, pred_dir, iou_threshold=0.5, confidence_threshold=0.01):
        """
        Initialize the OBB evaluator
        
        Args:
            gt_dir: Directory containing ground truth text files
            pred_dir: Directory containing prediction text files
            iou_threshold: IoU threshold for considering a detection as true positive
            confidence_threshold: Minimum confidence score to consider a prediction
        """
        self.gt_dir = gt_dir
        self.pred_dir = pred_dir
        self.iou_threshold = iou_threshold
        self.confidence_threshold = confidence_threshold
        self.class_names = set()
        self.results = {}
        # Store all predictions for PR curves
        self.all_predictions = defaultdict(list)
        # Store class cumulative stats
        self.class_cumulative = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        # Count of missing prediction files
        self.missing_pred_files = 0
    
    def parse_gt_file(self, gt_file):
        """
        Parse ground truth file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name difficulty_score
        """
        gts = []
        with open(gt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + difficulty
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and ignore difficulty score
                    class_name = parts[8]
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    gts.append({
                        'polygon': points,
                        'class': class_name,
                        'detected': False  # Flag to mark if this gt is detected
                    })
                except Exception as e:
                    print(f"Error parsing GT line: {line}")
                    print(f"Exception: {e}")
        return gts
    
    def parse_pred_file(self, pred_file):
        """
        Parse prediction file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name confidence_score
        """
        preds = []
        if not os.path.exists(pred_file):
            return preds  # Return empty list if file doesn't exist
            
        with open(pred_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + confidence
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and confidence score
                    class_name = parts[8]
                    confidence = float(parts[9])
                    
                    # Skip predictions with confidence below threshold
                    if confidence < self.confidence_threshold:
                        continue
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    preds.append({
                        'polygon': points,
                        'class': class_name,
                        'confidence': confidence
                    })
                except Exception as e:
                    print(f"Error parsing prediction line: {line}")
                    print(f"Exception: {e}")
        
        # Sort predictions by confidence (descending)
        preds.sort(key=lambda x: x['confidence'], reverse=True)
        return preds
    
    def calculate_iou(self, poly1, poly2):
        """Calculate IoU between two polygons"""
        try:
            polygon1 = Polygon(poly1)
            polygon2 = Polygon(poly2)
            
            if not polygon1.is_valid or not polygon2.is_valid:
                return 0.0
            
            # Calculate intersection and union areas
            intersection_area = polygon1.intersection(polygon2).area
            union_area = polygon1.area + polygon2.area - intersection_area
            
            # Avoid division by zero
            if union_area == 0:
                return 0.0
                
            return intersection_area / union_area
        except Exception as e:
            print(f"Error calculating IoU: {e}")
            return 0.0
    
    def evaluate_image(self, gt_file, pred_file):
        """Evaluate a single image"""
        image_name = os.path.basename(gt_file).split('.')[0]
        
        # Parse GT and prediction files
        gts = self.parse_gt_file(gt_file)
        preds = self.parse_pred_file(pred_file)
        
        # Check if prediction file exists
        if not os.path.exists(pred_file):
            self.missing_pred_files += 1
            # Count all GTs as false negatives if no predictions
            for gt in gts:
                self.class_cumulative[gt['class']]['FN'] += 1
            return
        
        # Initialize per-class results for this image
        class_results = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        class_predictions = defaultdict(list)
        
        # For each prediction, find the best matching ground truth
        for pred in preds:
            pred_class = pred['class']
            pred_poly = pred['polygon']
            pred_conf = pred['confidence']
            
            best_iou = 0
            best_gt_idx = -1
            
            # Find the best matching GT of the same class
            for idx, gt in enumerate(gts):
                if gt['class'] != pred_class or gt['detected']:
                    continue
                
                iou = self.calculate_iou(pred_poly, gt['polygon'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            # Record as TP or FP based on IoU threshold
            if best_gt_idx >= 0 and best_iou >= self.iou_threshold:
                class_predictions[pred_class].append((pred_conf, 1))  # True positive
                gts[best_gt_idx]['detected'] = True
                class_results[pred_class]['TP'] += 1
                self.class_cumulative[pred_class]['TP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 1))
            else:
                class_predictions[pred_class].append((pred_conf, 0))  # False positive
                class_results[pred_class]['FP'] += 1
                self.class_cumulative[pred_class]['FP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 0))
        
        # Count false negatives (undetected GTs)
        for gt in gts:
            if not gt['detected']:
                class_results[gt['class']]['FN'] += 1
                self.class_cumulative[gt['class']]['FN'] += 1
    
    def calculate_precision_recall(self, tp, fp, fn):
        """Calculate precision and recall"""
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        return precision, recall
    
    def calculate_f1(self, precision, recall):
        """Calculate F1 score"""
        return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    def calculate_ap(self, precisions, recalls):
        """Calculate Average Precision using 11-point interpolation"""
        ap = 0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11
        return ap
    
    def evaluate(self):
        """Evaluate all images and calculate metrics"""
        # Get list of files
        gt_files = sorted([f for f in os.listdir(self.gt_dir) if f.endswith('.txt')])
        print(f"Found {len(gt_files)} ground truth files")
        
        # Process each image
        for gt_file in gt_files:
            image_id = os.path.splitext(gt_file)[0]
            pred_file = image_id + '.txt'
            
            gt_path = os.path.join(self.gt_dir, gt_file)
            pred_path = os.path.join(self.pred_dir, pred_file)
            
            # Skip if prediction file doesn't exist but still record the missing files
            if not os.path.exists(pred_path):
                self.evaluate_image(gt_path, pred_path)  # This will handle missing files
                continue
            
            # Evaluate the image
            self.evaluate_image(gt_path, pred_path)
            
        if self.missing_pred_files > 0:
            print(f"\nWarning: {self.missing_pred_files} out of {len(gt_files)} prediction files were missing.")
            print(f"These are counted as having no detections (all ground truths counted as false negatives).")
        
        # Calculate metrics for each class
        self.results = {}
        mean_ap = 0
        processed_classes = 0
        
        for class_name in self.class_names:
            TP = self.class_cumulative[class_name]['TP']
            FP = self.class_cumulative[class_name]['FP']
            FN = self.class_cumulative[class_name]['FN']
            
            precision, recall = self.calculate_precision_recall(TP, FP, FN)
            f1 = self.calculate_f1(precision, recall)
            
            # Sort predictions by confidence for AP calculation
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            # Skip AP calculation if no ground truths or predictions
            if TP + FN == 0 or len(class_preds) == 0:
                ap = 0
            else:
                cumulative_tp = 0
                cumulative_fp = 0
                precisions = []
                recalls = []
                
                for conf, is_tp in class_preds:
                    if is_tp:
                        cumulative_tp += 1
                    else:
                        cumulative_fp += 1
                    
                    prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                    rec = cumulative_tp / (TP + FN)
                    
                    precisions.append(prec)
                    recalls.append(rec)
                
                precisions = np.array(precisions)
                recalls = np.array(recalls)
                ap = self.calculate_ap(precisions, recalls)
                processed_classes += 1
            
            self.results[class_name] = {
                'AP': ap,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'TP': TP,
                'FP': FP,
                'FN': FN
            }
            
            mean_ap += ap
        
        # Calculate mAP only over classes with predictions
        if processed_classes > 0:
            mean_ap /= processed_classes
        else:
            mean_ap = 0
        
        self.results['mAP'] = mean_ap
        
        return self.results
    
    def plot_precision_recall_curves(self, output_dir):
        """Plot precision-recall curves for all classes"""
        os.makedirs(output_dir, exist_ok=True)
        
        plt.figure(figsize=(10, 8))
        
        for class_name in sorted(self.class_names):
            # Skip classes with no predictions
            if class_name not in self.all_predictions or len(self.all_predictions[class_name]) == 0:
                continue
            
            # Sort predictions by confidence
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            TP = self.class_cumulative[class_name]['TP']
            FN = self.class_cumulative[class_name]['FN']
            
            # Skip if no ground truths
            if TP + FN == 0:
                continue
            
            # Calculate cumulative precision and recall
            cumulative_tp = 0
            cumulative_fp = 0
            precisions = []
            recalls = []
            
            for conf, is_tp in class_preds:
                if is_tp:
                    cumulative_tp += 1
                else:
                    cumulative_fp += 1
                
                prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                rec = cumulative_tp / (TP + FN)
                
                precisions.append(prec)
                recalls.append(rec)
            
            plt.plot(recalls, precisions, label=f"{class_name} (AP={self.results[class_name]['AP']:.3f})")
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'precision_recall_curves_RFRCNN.png'))
        plt.close()
    
    def print_results(self):
        """Print evaluation results"""
        print("\n===== EVALUATION RESULTS =====")
        print(f"IoU Threshold: {self.iou_threshold}")
        print(f"Confidence Threshold: {self.confidence_threshold}")
        print(f"mAP: {self.results['mAP']:.4f}")
        print("\nPer-class results:")
        
        # Print header
        print(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}")
        print('-' * 70)
        
        # Sort classes for consistent output
        for class_name in sorted(self.class_names):
            r = self.results[class_name]
            print(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}")
        
        print("=" * 70)
    
    def save_results_to_file(self, output_path):
        """Save results to file"""
        with open(output_path, 'w') as f:
            f.write("===== EVALUATION RESULTS =====\n")
            f.write(f"IoU Threshold: {self.iou_threshold}\n")
            f.write(f"Confidence Threshold: {self.confidence_threshold}\n")
            f.write(f"mAP: {self.results['mAP']:.4f}\n\n")
            
            # if self.missing_pred_files > 0:
            #     f.write(f"Note: {self.missing_pred_files} prediction files were missing and counted as no detections.\n\n")
                
            f.write("Per-class results:\n")
            
            # Print header
            f.write(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}\n")
            f.write('-' * 70 + '\n')
            
            # Sort classes for consistent output
            for class_name in sorted(self.class_names):
                r = self.results[class_name]
                f.write(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}\n")
            
            f.write("=" * 70 + '\n')


# Usage example
if __name__ == "__main__":
    # Define paths
    GT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\labels\train\labelTxt-v1.0\labelTxt"  # Ground truth directory
    PRED_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_rfrcnn\txt_annotations"  # Prediction directory
    OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\evaluation_results"  # Output directory
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Create evaluator with default parameters
    evaluator = OBBEvaluator(
        gt_dir=GT_DIR,
        pred_dir=PRED_DIR,
        iou_threshold=0.5,  # IoU threshold for TP/FP determination
        confidence_threshold=0.01  # Min confidence to consider a prediction
    )
    
    # Run evaluation
    results = evaluator.evaluate()
    
    # Print results
    evaluator.print_results()
    
    # Save results to file
    evaluator.save_results_to_file(os.path.join(OUTPUT_DIR, "evaluation_results_RFRCNN.txt"))
    
    # Generate PR curves
    try:
        evaluator.plot_precision_recall_curves(OUTPUT_DIR)
        print(f"Precision-Recall curves saved to {OUTPUT_DIR}")
    except Exception as e:
        print(f"Error generating PR curves: {e}")
    
    print(f"Evaluation completed! Results saved to {OUTPUT_DIR}")

Found 1411 ground truth files

===== EVALUATION RESULTS =====
IoU Threshold: 0.5
Confidence Threshold: 0.01
mAP: 0.3904

Per-class results:
Class           AP       Precision  Recall     F1       TP     FP     FN    
----------------------------------------------------------------------
baseball-diamond 0.4481   0.7802     0.4361     0.5595   181    51     234   
basketball-court 0.5397   0.6184     0.5883     0.6030   303    187    212   
bridge          0.0718   0.2895     0.0796     0.1249   163    400    1884  
ground-track-field 0.4461   0.6651     0.4400     0.5296   143    72     182   
harbor          0.3819   0.5595     0.4301     0.4863   2573   2026   3410  
helicopter      0.0909   0.1826     0.0635     0.0942   40     179    590   
large-vehicle   0.5940   0.7201     0.6231     0.6681   10574  4111   6395  
plane           0.5367   0.8055     0.5500     0.6536   4430   1070   3625  
roundabout      0.2381   0.3797     0.2807     0.3228   112    183    287   
ship          

# S2ANET

In [5]:
import os
import cv2
import torch
import mmrotate
import numpy as np
from mmdet.apis import init_detector, inference_detector

# -------------------- Path Setup --------------------
IMAGE_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\images\train\images"
OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_s2anet"
CONFIG_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\mmrotate\configs\s2anet\s2anet_r50_fpn_fp16_1x_dota_le135.py"
CHECKPOINT_FILE = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\s2anet_r50_fpn_fp16_1x_dota_le135-5cac515c.pth"
LOW_CONFIDENCE = 0.1

os.makedirs(OUTPUT_DIR, exist_ok=True)
TXT_FOLDER = os.path.join(OUTPUT_DIR, "txt_annotations")
os.makedirs(TXT_FOLDER, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Using device: {device}")

model = init_detector(CONFIG_FILE, CHECKPOINT_FILE, device=device)
class_names = model.CLASSES if hasattr(model, 'CLASSES') else model.dataset_meta['CLASSES']
print(f"✅ Model loaded with {len(class_names)} classes: {class_names}")

def draw_obb(img, obb, label, score, color=(0, 255, 0), thickness=2):
    cx, cy, w, h, angle = obb
    rect = ((cx, cy), (w, h), angle * 180 / np.pi)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    cv2.drawContours(img, [box], 0, color, thickness)
    label_text = f"{class_names[label] if label < len(class_names) else 'No_Detection'}: {score:.2f}"
    cv2.putText(img, label_text, (int(cx), int(cy) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

# Gather image files
image_files = [f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
if not image_files:
    print(f"❌ No image files found in {IMAGE_DIR}")
else:
    print(f"✅ Found {len(image_files)} image(s) to process")

total_images = 0
images_with_detections = 0
skipped_images = []
class_detection_counts = {i: 0 for i in range(len(class_names))}
class_detection_counts["No_Detection"] = 0

for img_name in image_files:
    total_images += 1
    img_path = os.path.join(IMAGE_DIR, img_name)
    print(f"Processing {img_name}...")

    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Failed to load image: {img_path}")
        skipped_images.append(img_name)
        continue

    result = inference_detector(model, img_path)

    drawn_any = False
    pred_data = []

    if isinstance(result, tuple):
        bbox_results, _ = result
        for class_id, bboxes in enumerate(bbox_results):
            for bbox in bboxes:
                score = bbox[5]
                obb = bbox[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })
    elif hasattr(result, 'pred_instances'):
        pred_instances = result.pred_instances
        bboxes = pred_instances.bboxes.cpu().numpy()
        scores = pred_instances.scores.cpu().numpy()
        labels = pred_instances.labels.cpu().numpy()
        for bbox, score, label in zip(bboxes, scores, labels):
            obb = bbox.tolist()
            if score >= LOW_CONFIDENCE:
                draw_obb(img, obb, label, score)
                drawn_any = True
                class_detection_counts[label.item()] += 1
                pred_data.append({
                    "obb": obb,
                    "score": float(score),
                    "label": int(label),
                    "label_name": class_names[label]
                })
    else:
        for class_id, class_result in enumerate(result):
            for res in class_result:
                score = res[-1]
                obb = res[:5].tolist()
                if score >= LOW_CONFIDENCE:
                    draw_obb(img, obb, class_id, score)
                    drawn_any = True
                    class_detection_counts[class_id] += 1
                    pred_data.append({
                        "obb": obb,
                        "score": float(score),
                        "label": int(class_id),
                        "label_name": class_names[class_id]
                    })

    # If no detection, draw dummy red box
    if not drawn_any:
        print(f"⚠️ No predictions for {img_name}, drawing dummy box.")
        h, w = img.shape[:2]
        dummy_cx, dummy_cy = w // 2, h // 2
        dummy_w, dummy_h = w // 5, h // 5
        dummy_angle = 0
        dummy_obb = [dummy_cx, dummy_cy, dummy_w, dummy_h, dummy_angle]
        dummy_score = 0.0
        draw_obb(img, dummy_obb, len(class_names), dummy_score, color=(0, 0, 255))
        pred_data.append({
            "obb": dummy_obb,
            "score": dummy_score,
            "label": len(class_names),
            "label_name": "No_Detection"
        })
        class_detection_counts["No_Detection"] += 1
    else:
        print(f"✅ Predictions drawn for {img_name}")
        images_with_detections += 1

    # Save image
    output_path = os.path.join(OUTPUT_DIR, img_name)
    cv2.imwrite(output_path, img)

    # Save .txt annotation
    txt_filename = os.path.splitext(img_name)[0] + ".txt"
    txt_path = os.path.join(TXT_FOLDER, txt_filename)

    with open(txt_path, 'w') as f:
        for pred in pred_data:
            cx, cy, w, h, angle = pred['obb']
            rect = ((cx, cy), (w, h), angle * 180 / np.pi)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            points_flat = box.flatten()
            coords = " ".join(map(str, points_flat))
            label_name = pred['label_name'].replace(" ", "_")  # Prevent space in label
            score = pred['score']
            line = f"{coords} {label_name} {score:.4f}\n"
            f.write(line)

    print(f"📄 TXT saved: {txt_filename}")

# ========== Summary ==========
print("\n===== Detection Summary =====")
print(f"Total images processed: {total_images}")
print(f"Images with detections: {images_with_detections} ({images_with_detections / total_images * 100:.1f}%)")
print(f"Images with dummy boxes: {total_images - images_with_detections} ({(total_images - images_with_detections) / total_images * 100:.1f}%)")

print("\n📊 Detections by class:")
for class_id, count in class_detection_counts.items():
    name = class_names[class_id] if isinstance(class_id, int) and class_id < len(class_names) else class_id
    if count > 0:
        print(f"  {name}: {count} detections")

if skipped_images:
    print("\n⚠️ Skipped the following images (could not load):")
    for name in skipped_images:
        print(f" - {name}")

print("\n🎉 All images processed and saved with OBBs and TXT annotations.")

✅ Using device: cuda
load checkpoint from local path: D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\pth files\s2anet_r50_fpn_fp16_1x_dota_le135-5cac515c.pth
✅ Model loaded with 15 classes: ('plane', 'baseball-diamond', 'bridge', 'ground-track-field', 'small-vehicle', 'large-vehicle', 'ship', 'tennis-court', 'basketball-court', 'storage-tank', 'soccer-ball-field', 'roundabout', 'harbor', 'swimming-pool', 'helicopter')
✅ Found 1411 image(s) to process
Processing P0000.png...


d:\Anaconda\envs\cfi_net\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


✅ Predictions drawn for P0000.png
📄 TXT saved: P0000.txt
Processing P0001.png...
✅ Predictions drawn for P0001.png
📄 TXT saved: P0001.txt
Processing P0002.png...
✅ Predictions drawn for P0002.png
📄 TXT saved: P0002.txt
Processing P0005.png...
✅ Predictions drawn for P0005.png
📄 TXT saved: P0005.txt
Processing P0008.png...
✅ Predictions drawn for P0008.png
📄 TXT saved: P0008.txt
Processing P0010.png...
✅ Predictions drawn for P0010.png
📄 TXT saved: P0010.txt
Processing P0011.png...
✅ Predictions drawn for P0011.png
📄 TXT saved: P0011.txt
Processing P0012.png...
✅ Predictions drawn for P0012.png
📄 TXT saved: P0012.txt
Processing P0013.png...
✅ Predictions drawn for P0013.png
📄 TXT saved: P0013.txt
Processing P0018.png...
✅ Predictions drawn for P0018.png
📄 TXT saved: P0018.txt
Processing P0020.png...
✅ Predictions drawn for P0020.png
📄 TXT saved: P0020.txt
Processing P0021.png...
✅ Predictions drawn for P0021.png
📄 TXT saved: P0021.txt
Processing P0022.png...
✅ Predictions drawn for P002

In [2]:
import os
import numpy as np
import cv2
from shapely.geometry import Polygon
from collections import defaultdict
import matplotlib.pyplot as plt

class OBBEvaluator:
    def __init__(self, gt_dir, pred_dir, iou_threshold=0.5, confidence_threshold=0.01):
        """
        Initialize the OBB evaluator
        
        Args:
            gt_dir: Directory containing ground truth text files
            pred_dir: Directory containing prediction text files
            iou_threshold: IoU threshold for considering a detection as true positive
            confidence_threshold: Minimum confidence score to consider a prediction
        """
        self.gt_dir = gt_dir
        self.pred_dir = pred_dir
        self.iou_threshold = iou_threshold
        self.confidence_threshold = confidence_threshold
        self.class_names = set()
        self.results = {}
        # Store all predictions for PR curves
        self.all_predictions = defaultdict(list)
        # Store class cumulative stats
        self.class_cumulative = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        # Count of missing prediction files
        self.missing_pred_files = 0
    
    def parse_gt_file(self, gt_file):
        """
        Parse ground truth file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name difficulty_score
        """
        gts = []
        with open(gt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + difficulty
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and ignore difficulty score
                    class_name = parts[8]
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    gts.append({
                        'polygon': points,
                        'class': class_name,
                        'detected': False  # Flag to mark if this gt is detected
                    })
                except Exception as e:
                    print(f"Error parsing GT line: {line}")
                    print(f"Exception: {e}")
        return gts
    
    def parse_pred_file(self, pred_file):
        """
        Parse prediction file with format:
        x1 y1 x2 y2 x3 y3 x4 y4 class_name confidence_score
        """
        preds = []
        if not os.path.exists(pred_file):
            return preds  # Return empty list if file doesn't exist
            
        with open(pred_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 10:  # Need at least 8 coordinates + class + confidence
                    continue
                try:
                    # Extract coordinates as integers
                    coords = list(map(int, parts[:8]))
                    # Extract class name and confidence score
                    class_name = parts[8]
                    confidence = float(parts[9])
                    
                    # Skip predictions with confidence below threshold
                    if confidence < self.confidence_threshold:
                        continue
                    
                    # Add class to known classes
                    self.class_names.add(class_name)
                    
                    # Create polygon from coordinates
                    points = np.array(coords).reshape(-1, 2)
                    
                    preds.append({
                        'polygon': points,
                        'class': class_name,
                        'confidence': confidence
                    })
                except Exception as e:
                    print(f"Error parsing prediction line: {line}")
                    print(f"Exception: {e}")
        
        # Sort predictions by confidence (descending)
        preds.sort(key=lambda x: x['confidence'], reverse=True)
        return preds
    
    def calculate_iou(self, poly1, poly2):
        """Calculate IoU between two polygons"""
        try:
            polygon1 = Polygon(poly1)
            polygon2 = Polygon(poly2)
            
            if not polygon1.is_valid or not polygon2.is_valid:
                return 0.0
            
            # Calculate intersection and union areas
            intersection_area = polygon1.intersection(polygon2).area
            union_area = polygon1.area + polygon2.area - intersection_area
            
            # Avoid division by zero
            if union_area == 0:
                return 0.0
                
            return intersection_area / union_area
        except Exception as e:
            print(f"Error calculating IoU: {e}")
            return 0.0
    
    def evaluate_image(self, gt_file, pred_file):
        """Evaluate a single image"""
        image_name = os.path.basename(gt_file).split('.')[0]
        
        # Parse GT and prediction files
        gts = self.parse_gt_file(gt_file)
        preds = self.parse_pred_file(pred_file)
        
        # Check if prediction file exists
        if not os.path.exists(pred_file):
            self.missing_pred_files += 1
            # Count all GTs as false negatives if no predictions
            for gt in gts:
                self.class_cumulative[gt['class']]['FN'] += 1
            return
        
        # Initialize per-class results for this image
        class_results = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})
        class_predictions = defaultdict(list)
        
        # For each prediction, find the best matching ground truth
        for pred in preds:
            pred_class = pred['class']
            pred_poly = pred['polygon']
            pred_conf = pred['confidence']
            
            best_iou = 0
            best_gt_idx = -1
            
            # Find the best matching GT of the same class
            for idx, gt in enumerate(gts):
                if gt['class'] != pred_class or gt['detected']:
                    continue
                
                iou = self.calculate_iou(pred_poly, gt['polygon'])
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            # Record as TP or FP based on IoU threshold
            if best_gt_idx >= 0 and best_iou >= self.iou_threshold:
                class_predictions[pred_class].append((pred_conf, 1))  # True positive
                gts[best_gt_idx]['detected'] = True
                class_results[pred_class]['TP'] += 1
                self.class_cumulative[pred_class]['TP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 1))
            else:
                class_predictions[pred_class].append((pred_conf, 0))  # False positive
                class_results[pred_class]['FP'] += 1
                self.class_cumulative[pred_class]['FP'] += 1
                # Store for PR curve
                self.all_predictions[pred_class].append((pred_conf, 0))
        
        # Count false negatives (undetected GTs)
        for gt in gts:
            if not gt['detected']:
                class_results[gt['class']]['FN'] += 1
                self.class_cumulative[gt['class']]['FN'] += 1
    
    def calculate_precision_recall(self, tp, fp, fn):
        """Calculate precision and recall"""
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        return precision, recall
    
    def calculate_f1(self, precision, recall):
        """Calculate F1 score"""
        return 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    def calculate_ap(self, precisions, recalls):
        """Calculate Average Precision using 11-point interpolation"""
        ap = 0
        for t in np.arange(0, 1.1, 0.1):
            if np.sum(recalls >= t) == 0:
                p = 0
            else:
                p = np.max(precisions[recalls >= t])
            ap += p / 11
        return ap
    
    def evaluate(self):
        """Evaluate all images and calculate metrics"""
        # Get list of files
        gt_files = sorted([f for f in os.listdir(self.gt_dir) if f.endswith('.txt')])
        print(f"Found {len(gt_files)} ground truth files")
        
        # Process each image
        for gt_file in gt_files:
            image_id = os.path.splitext(gt_file)[0]
            pred_file = image_id + '.txt'
            
            gt_path = os.path.join(self.gt_dir, gt_file)
            pred_path = os.path.join(self.pred_dir, pred_file)
            
            # Skip if prediction file doesn't exist but still record the missing files
            if not os.path.exists(pred_path):
                self.evaluate_image(gt_path, pred_path)  # This will handle missing files
                continue
            
            # Evaluate the image
            self.evaluate_image(gt_path, pred_path)
            
        if self.missing_pred_files > 0:
            print(f"\nWarning: {self.missing_pred_files} out of {len(gt_files)} prediction files were missing.")
            print(f"These are counted as having no detections (all ground truths counted as false negatives).")
        
        # Calculate metrics for each class
        self.results = {}
        mean_ap = 0
        processed_classes = 0
        
        for class_name in self.class_names:
            TP = self.class_cumulative[class_name]['TP']
            FP = self.class_cumulative[class_name]['FP']
            FN = self.class_cumulative[class_name]['FN']
            
            precision, recall = self.calculate_precision_recall(TP, FP, FN)
            f1 = self.calculate_f1(precision, recall)
            
            # Sort predictions by confidence for AP calculation
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            # Skip AP calculation if no ground truths or predictions
            if TP + FN == 0 or len(class_preds) == 0:
                ap = 0
            else:
                cumulative_tp = 0
                cumulative_fp = 0
                precisions = []
                recalls = []
                
                for conf, is_tp in class_preds:
                    if is_tp:
                        cumulative_tp += 1
                    else:
                        cumulative_fp += 1
                    
                    prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                    rec = cumulative_tp / (TP + FN)
                    
                    precisions.append(prec)
                    recalls.append(rec)
                
                precisions = np.array(precisions)
                recalls = np.array(recalls)
                ap = self.calculate_ap(precisions, recalls)
                processed_classes += 1
            
            self.results[class_name] = {
                'AP': ap,
                'Precision': precision,
                'Recall': recall,
                'F1': f1,
                'TP': TP,
                'FP': FP,
                'FN': FN
            }
            
            mean_ap += ap
        
        # Calculate mAP only over classes with predictions
        if processed_classes > 0:
            mean_ap /= processed_classes
        else:
            mean_ap = 0
        
        self.results['mAP'] = mean_ap
        
        return self.results
    
    def plot_precision_recall_curves(self, output_dir):
        """Plot precision-recall curves for all classes"""
        os.makedirs(output_dir, exist_ok=True)
        
        plt.figure(figsize=(10, 8))
        
        for class_name in sorted(self.class_names):
            # Skip classes with no predictions
            if class_name not in self.all_predictions or len(self.all_predictions[class_name]) == 0:
                continue
            
            # Sort predictions by confidence
            class_preds = sorted(self.all_predictions[class_name], key=lambda x: x[0], reverse=True)
            
            TP = self.class_cumulative[class_name]['TP']
            FN = self.class_cumulative[class_name]['FN']
            
            # Skip if no ground truths
            if TP + FN == 0:
                continue
            
            # Calculate cumulative precision and recall
            cumulative_tp = 0
            cumulative_fp = 0
            precisions = []
            recalls = []
            
            for conf, is_tp in class_preds:
                if is_tp:
                    cumulative_tp += 1
                else:
                    cumulative_fp += 1
                
                prec = cumulative_tp / (cumulative_tp + cumulative_fp)
                rec = cumulative_tp / (TP + FN)
                
                precisions.append(prec)
                recalls.append(rec)
            
            plt.plot(recalls, precisions, label=f"{class_name} (AP={self.results[class_name]['AP']:.3f})")
        
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(output_dir, 'precision_recall_curves_S2ANET.png'))
        plt.close()
    
    def print_results(self):
        """Print evaluation results"""
        print("\n===== EVALUATION RESULTS =====")
        print(f"IoU Threshold: {self.iou_threshold}")
        print(f"Confidence Threshold: {self.confidence_threshold}")
        print(f"mAP: {self.results['mAP']:.4f}")
        print("\nPer-class results:")
        
        # Print header
        print(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}")
        print('-' * 70)
        
        # Sort classes for consistent output
        for class_name in sorted(self.class_names):
            r = self.results[class_name]
            print(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}")
        
        print("=" * 70)
    
    def save_results_to_file(self, output_path):
        """Save results to file"""
        with open(output_path, 'w') as f:
            f.write("===== EVALUATION RESULTS =====\n")
            f.write(f"IoU Threshold: {self.iou_threshold}\n")
            f.write(f"Confidence Threshold: {self.confidence_threshold}\n")
            f.write(f"mAP: {self.results['mAP']:.4f}\n\n")
            
            # if self.missing_pred_files > 0:
            #     f.write(f"Note: {self.missing_pred_files} prediction files were missing and counted as no detections.\n\n")
                
            f.write("Per-class results:\n")
            
            # Print header
            f.write(f"{'Class':<15} {'AP':<8} {'Precision':<10} {'Recall':<10} {'F1':<8} {'TP':<6} {'FP':<6} {'FN':<6}\n")
            f.write('-' * 70 + '\n')
            
            # Sort classes for consistent output
            for class_name in sorted(self.class_names):
                r = self.results[class_name]
                f.write(f"{class_name:<15} {r['AP']:.4f}   {r['Precision']:.4f}     {r['Recall']:.4f}     {r['F1']:.4f}   {r['TP']:<6} {r['FP']:<6} {r['FN']:<6}\n")
            
            f.write("=" * 70 + '\n')


# Usage example
if __name__ == "__main__":
    # Define paths
    GT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\DOTA V1\labels\train\labelTxt-v1.0\labelTxt"  # Ground truth directory
    PRED_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\output_s2anet\txt_annotations"  # Prediction directory
    OUTPUT_DIR = r"D:\AU MATERIAL\2 SEM\CV\PROJECT\codes\evaluation_results"  # Output directory
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Create evaluator with default parameters
    evaluator = OBBEvaluator(
        gt_dir=GT_DIR,
        pred_dir=PRED_DIR,
        iou_threshold=0.5,  # IoU threshold for TP/FP determination
        confidence_threshold=0.01  # Min confidence to consider a prediction
    )
    
    # Run evaluation
    results = evaluator.evaluate()
    
    # Print results
    evaluator.print_results()
    
    # Save results to file
    evaluator.save_results_to_file(os.path.join(OUTPUT_DIR, "evaluation_results_S2ANET.txt"))
    
    # Generate PR curves
    try:
        evaluator.plot_precision_recall_curves(OUTPUT_DIR)
        print(f"Precision-Recall curves saved to {OUTPUT_DIR}")
    except Exception as e:
        print(f"Error generating PR curves: {e}")
    
    print(f"Evaluation completed! Results saved to {OUTPUT_DIR}")

Found 1411 ground truth files

===== EVALUATION RESULTS =====
IoU Threshold: 0.5
Confidence Threshold: 0.01
mAP: 0.3786

Per-class results:
Class           AP       Precision  Recall     F1       TP     FP     FN    
----------------------------------------------------------------------
baseball-diamond 0.4430   0.5248     0.4337     0.4749   180    163    235   
basketball-court 0.5298   0.2052     0.5709     0.3018   294    1139   221   
bridge          0.0909   0.1502     0.0874     0.1105   179    1013   1868  
ground-track-field 0.4113   0.3820     0.4185     0.3994   136    220    189   
harbor          0.3127   0.3591     0.3321     0.3451   1987   3546   3996  
helicopter      0.0909   0.0255     0.0540     0.0347   34     1298   596   
large-vehicle   0.5741   0.5208     0.6070     0.5606   10300  9479   6669  
plane           0.4385   0.5548     0.4651     0.5060   3746   3006   4309  
roundabout      0.2725   0.2092     0.3083     0.2492   123    465    276   
ship          